<a href="https://colab.research.google.com/github/sandeep92134/PYTHON-Data-Cleaning/blob/master/Chapter%206/Exersize%204.%20Changing%20series%20values%20conditionally.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
nls97 = pd.read_csv("https://raw.githubusercontent.com/sandeep92134/PYTHON-Data-Cleaning/master/Chapter%206/datasets/nls97b.csv")
nls97.set_index("personid", inplace=True)
landtemps = pd.read_csv("https://raw.githubusercontent.com/sandeep92134/PYTHON-Data-Cleaning/master/Chapter%206/datasets/landtemps2019avgs.csv")

In [2]:
landtemps.elevation.quantile(np.arange(0.2,1.1,0.2))

0.2      48.0
0.4     190.5
0.6     393.2
0.8    1066.8
1.0    9999.0
Name: elevation, dtype: float64

In [3]:
landtemps['elevation_group'] = np.where(landtemps.elevation>landtemps.elevation.quantile(0.8),'High','Low')
landtemps.elevation_group = landtemps.elevation_group.astype('category')
landtemps.groupby(['elevation_group'])['elevation'].\
agg(['count','min','max'])

,count,min,max
elevation_group,,,
High,2409,1067.0,9999.0
Low,9686,-350.0,1066.8


In [4]:
landtemps.elevation.median()

271.3

In [5]:
landtemps['elevation_group'] = np.where(landtemps.elevation>
landtemps.elevation.quantile(0.8),'High',np.where(landtemps.elevation>
landtemps.elevation.median(),'Medium','Low'))
landtemps.elevation_group = landtemps.elevation_group.astype('category')
landtemps.groupby(['elevation_group'])['elevation'].agg(['count','min','max'])

,count,min,max
elevation_group,,,
High,2409,1067.0,9999.0
Low,6056,-350.0,271.3
Medium,3630,271.4,1066.8


In [6]:
test = [(nls97.gpaoverall<2) & (nls97.highestdegree=='0. None'), nls97.highestdegree=='0. None', nls97.gpaoverall<2]
result = ['1. Low GPA and No Diploma','2. No Diploma','3. Low GPA']
nls97['hsachieve'] = np.select(test, result, '4. Did Okay')
nls97[['hsachieve','gpaoverall','highestdegree']].head()

,hsachieve,gpaoverall,highestdegree
personid,,,
100061,4. Did Okay,3.06,2. High School
100139,4. Did Okay,NaN,2. High School
100284,2. No Diploma,NaN,0. None
100292,4. Did Okay,3.45,4. Bachelors
100583,4. Did Okay,2.91,2. High School


In [7]:
nls97.hsachieve.value_counts().sort_index()

1. Low GPA and No Diploma      95
2. No Diploma                 858
3. Low GPA                    459
4. Did Okay                  7572
Name: hsachieve, dtype: int64

In [8]:
nls97.loc[[100292,100583,100139], 'colenrfeb00':'colenroct04'].T

personid,100292,100583,100139
colenrfeb00,1. Not enrolled,1. Not enrolled,1. Not enrolled
colenroct00,3. 4-year college,1. Not enrolled,1. Not enrolled
colenrfeb01,3. 4-year college,1. Not enrolled,1. Not enrolled
colenroct01,3. 4-year college,3. 4-year college,1. Not enrolled
colenrfeb02,3. 4-year college,3. 4-year college,1. Not enrolled
colenroct02,3. 4-year college,1. Not enrolled,1. Not enrolled
colenrfeb03,3. 4-year college,1. Not enrolled,1. Not enrolled
colenroct03,3. 4-year college,1. Not enrolled,1. Not enrolled
colenrfeb04,3. 4-year college,1. Not enrolled,1. Not enrolled
colenroct04,1. Not enrolled,1. Not enrolled,1. Not enrolled


In [13]:
nls97['baenrollment'] = nls97.filter(like="colenr").\
  apply(lambda x: x.str[0:1]=='3').\
  any(axis=1)

In [14]:
nls97.loc[[100292,100583,100139], ['baenrollment']].T

personid,100292,100583,100139
baenrollment,True,True,False


In [16]:
nls97.baenrollment.value_counts()

False    5085
True     3899
Name: baenrollment, dtype: int64

In [17]:
# use apply and lambda to create a more complicated categorical series
def getsleepdeprivedreason(row):
  sleepdeprivedreason = "Unknown"
  if (row.nightlyhrssleep>=6):
    sleepdeprivedreason = "Not Sleep Deprived"
  elif (row.nightlyhrssleep>0):
    if (row.weeksworked16+row.weeksworked17 < 80):
      if (row.childathome>2):
        sleepdeprivedreason = "Child Rearing"
      else:
        sleepdeprivedreason = "Other Reasons"
    else:
      if (row.wageincome>=62000 or row.highestgradecompleted>=16):
        sleepdeprivedreason = "Work Pressure"
      else:
        sleepdeprivedreason = "Income Pressure"
  else:
    sleepdeprivedreason = "Unknown"
  return sleepdeprivedreason

In [18]:
nls97['sleepdeprivedreason'] = nls97.apply(getsleepdeprivedreason, axis=1)
nls97.sleepdeprivedreason = nls97.sleepdeprivedreason.astype('category')
nls97.sleepdeprivedreason.value_counts()

Not Sleep Deprived    5595
Unknown               2286
Income Pressure        462
Work Pressure          281
Other Reasons          272
Child Rearing           88
Name: sleepdeprivedreason, dtype: int64